<a href="https://colab.research.google.com/github/federicOO1/LAB-IA/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import librerie


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets
import matplotlib.pyplot as plt
from torchvision import transforms
import torch.nn as nn
import os
import numpy as np
import PIL
import warnings
from tqdm import tqdm
import torch.optim as optim
import albumentations
warnings.filterwarnings('ignore')

In [2]:
!pip install rasterio
import rasterio
from rasterio.plot import reshape_as_image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 55.2 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.chdir("/content/drive/MyDrive/PotsdamDataset")

In [43]:
class PotsdamDataset(Dataset):
    def __init__(self, dataset_folder, transform=None):
        self.dataset_folder = dataset_folder
        self.image_paths = []
        self.world_file_paths = []
        self.mask_paths = []
        self.transform = transform

        # Leggi i percorsi delle immagini, dei file .tfw e delle maschere
        end_RGBIR_folder = os.listdir(dataset_folder)[0]
        end_LABELS_folder = os.listdir(dataset_folder)[1]

        RGBIR_folder = data_folder + '/' + end_RGBIR_folder
        LABELS_folder = data_folder + '/' + end_LABELS_folder

        for file_name in os.listdir(RGBIR_folder):
            if file_name.endswith('.tif'):
                image_path = os.path.join(RGBIR_folder, file_name)
                world_file_path = os.path.join(RGBIR_folder, file_name.replace('.tif', '.tfw'))

                if os.path.exists(world_file_path):
                    self.image_paths.append(image_path)
                    self.world_file_paths.append(world_file_path)


        for label_name in os.listdir(LABELS_folder):
            mask_path = os.path.join(LABELS_folder, label_name)

            if os.path.exists(mask_path):
              self.mask_paths.append(mask_path)
        self.image_paths.sort()
        self.world_file_paths.sort()
        self.mask_paths.sort()

    def __len__(self):
        return len(self.image_paths)

    def get_image_paths(self, indices):
        return [self.image_paths[idx] for idx in indices]

    def get_mask_paths(self, indices):
        return [self.mask_paths[idx] for idx in indices]

    def load_world_file(self, world_file_path):
          lines = open(world_file_path).readlines()
          try:
              parameters = [float(line.strip()) for line in lines if line.strip()]
              if len(parameters) == 6:
                  return parameters
              else:
                  raise ValueError("Il file .tfw non contiene 6 parametri.")
          except Exception as e:
              print(f"Errore durante la lettura dei parametri di georeferenziazione: {str(e)}")
              return None

    def convert_labels_to_tensor(self, rgb_label):
      #print("Current rgb_label:",rgb_label)
      # Definisci i colori delle classi nella maschera RGB
      colors_to_labels = {
          (255, 255, 255): 0,  # Impervious surfaces -> Classe 0
          (0, 0, 255): 1,  # Building -> Classe 1
          (0, 255, 255): 2,  # Low vegetation -> Classe 2
          (0, 255, 0): 3,  # Tree -> Classe 3
          (255, 255, 0): 4,  # Car -> Classe 4
          (255, 0, 0): 5  # Clutter/background -> Classe 5
      }
      color_image = np.array(rgb_label)

      # Rappresenta color_array come un array (6, 3)
      color_array = np.array(list(colors_to_labels.keys()),dtype=np.uint8)

      # Reshape l'immagine per renderla compatibile con l'operazione di broadcasting
      color_image_reshaped = color_image.reshape(-1, 3)

      # Calcola le differenze tra i colori nella maschera RGB e i colori delle classi
      color_diffs = np.sum(np.abs(color_image_reshaped[:, None, :] - color_array[None, :, :]), axis=-1)

      # Trova l'indice del colore più vicino per ciascun pixel
      closest_color_indices = np.argmin(color_diffs, axis=-1)

      # Mappa gli indici di colore ai valori delle classi
      class_label = np.where(np.min(color_diffs, axis=-1) == 0, closest_color_indices, -1)

      # Reshape per tornare alle dimensioni dell'immagine originale
      class_label = class_label.reshape(rgb_label.shape[1], rgb_label.shape[2])

      # Crea il tensore delle etichette di classe
      class_label_tensor = torch.tensor(class_label, dtype=torch.long)

      return class_label_tensor

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        world_file_path = self.world_file_paths[idx]
        mask_path = self.mask_paths[idx]

        # Carica l'immagine TIFF utilizzando la libreria rasterio
        image = rasterio.open(image_path).read()

        # Carica i parametri di georeferenziazione dal file .tfw
        world_params = self.load_world_file(world_file_path)

        # Carica la maschera con rasterio
        mask = rasterio.open(mask_path).read()

        # Converti la maschera RGB nel formato appropriato per CrossEntropyLoss
        label = self.convert_labels_to_tensor(mask)

        image = torch.from_numpy(image)

        # Estrai mean e std dalla lista di trasformazione, se presente
        if self.transform and isinstance(self.transform, list) and len(self.transform) == 2:
            mean, std = self.transform
            # Applica la trasformazione di normalizzazione se sono presenti mean e std
            image = self.normalize(image, mean, std)


        # Restituisci l'immagine, la maschera e i parametri di georeferenziazione
        return image, label, world_params

    def normalize(self, image, mean=None, std=None):
        if mean is not None and std is not None:
            # Converte l'immagine in float
            image = image.float()

            # Normalizza per canale
            for c in range(image.size(0)):
                image[c] = (image[c] - mean[c]) / std[c]

        return image

In [13]:
# Definisci il percorso della cartella contenente i dati .tif e .tfw
data_folder = "/content/drive/MyDrive/PotsdamDataset"

# Crea un'istanza del dataset
dataset = PotsdamDataset(data_folder)

size = len(dataset)
train_size = int(0.7 * size)
val_size = int(0.15 * size)
test_size = int(size - train_size - val_size)

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

batch_size = 2

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
##
num_channels = 4  # RGBIR has 4 channels
dtype = torch.float32  # Imposta il tipo di dato desiderato

# placeholders
psum = torch.zeros(num_channels, dtype=dtype)
psum_sq = torch.zeros(num_channels, dtype=dtype)
count = 0

# loop through images
for inputs, labels, georeference_info in tqdm(train_loader):
    # Converti il tensore in tipo di dato a precisione maggiore
    inputs = inputs.to(dtype)

    psum += inputs.sum(axis = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

    # Conteggio dei pixel
    count += inputs.size(0) * inputs.size(2) * inputs.size(3)

total_mean = psum / count

# Calcola la varianza per canale
total_var = (psum_sq / count) - (total_mean ** 2)

# Calcola la deviazione standard per canale
total_std = torch.sqrt(total_var)
# output
print('mean per channel: ' + str(total_mean))
print('std per channel: ' + str(total_std))


100%|██████████| 13/13 [09:20<00:00, 43.14s/it]

mean per channel: tensor([86.8845, 92.9506, 86.3022, 98.7621])
std per channel: tensor([35.9432, 35.8858, 37.4310, 36.1334])


In [29]:
normalize_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=total_mean.tolist(), std=total_std.tolist())])

In [44]:
dataset_with_transform = PotsdamDataset(data_folder, transform=[total_mean, total_std])

In [45]:
dataset_with_transform[0][0]

tensor([[[-0.8314, -0.8314, -0.9427,  ..., -0.2750, -0.4141, -0.5810],
         [-1.0262, -0.9984, -0.9705,  ..., -0.3028, -0.3585, -0.6089],
         [-1.1653, -1.0540, -1.0262,  ..., -0.2472, -0.3585, -0.6089],
         ...,
         [ 0.4484,  0.5318,  0.6431,  ...,  0.0589,  0.0589,  0.0589],
         [ 0.3649,  0.4484,  0.4205,  ...,  0.0589,  0.0589,  0.0589],
         [ 0.3371,  0.4205,  0.4205,  ...,  0.0032,  0.0310,  0.0589]],

        [[-0.7510, -0.7510, -0.8903,  ..., -0.2494, -0.3887, -0.6117],
         [-1.0018, -0.9182, -0.9461,  ..., -0.2773, -0.3052, -0.6674],
         [-1.1969, -0.9739, -0.9461,  ..., -0.1658, -0.3052, -0.6395],
         ...,
         [ 0.6702,  0.7259,  0.7816,  ...,  0.5308,  0.5308,  0.5587],
         [ 0.6144,  0.5866,  0.5030,  ...,  0.5030,  0.5030,  0.5587],
         [ 0.5587,  0.5866,  0.4472,  ...,  0.4751,  0.5030,  0.6144]],

        [[-0.7027, -0.7027, -0.8363,  ..., -0.5424, -0.7027, -0.8095],
         [-0.9966, -0.9431, -0.9431,  ..., -0

UnidentifiedImageError: cannot identify image file <_io.BufferedReader name='/content/drive/MyDrive/PotsdamDataset/4_Ortho_RGBIR/top_potsdam_2_10_RGBIR.tif'>

##Implementazione UNET

In [11]:
import torch
import torch.nn as nn

class DoubleConvolution(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConvolution, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels, features=[64,128,256,512]):
        super(UNet, self).__init__()

        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        for feature in features:
          self.downs.append(DoubleConvolution(in_channels, feature))
          in_channels = feature

        for feature in reversed(features):
          self.ups.append(nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2))
          self.ups.append(DoubleConvolution(feature*2, feature))

        self.bottleneck = DoubleConvolution(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)


    def forward(self, x):

      skip_connections = []
      for down in self.downs:
        x = down(x)
        skip_connections.append(x)
        x = self.pool(x)

      x = self.bottleneck(x)
      skip_connections = skip_connections[::-1]

      for i in range(0,len(self.ups),2):
        x = self.ups[i](x)
        skip_connection = skip_connections[i//2]

        if x.shape != skip_connection.shape:
          x = transforms.functional.resize(x, size=skip_connection.shape[2:])

        concat_skip = torch.cat((skip_connection,x), dim=1)
        x = self.ups[i+1](concat_skip)

      return self.final_conv(x)



In [ ]:
x = torch.randn((3,1,160,160))
model = UNet(in_channels=1, out_channels=1)
preds = model(x)
print(preds.shape)
print(x.shape)
assert preds.shape == x.shape

torch.Size([3, 1, 160, 160])
torch.Size([3, 1, 160, 160])


##Training part

In [12]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def train_function(loader, model, optimizer, loss_function, scaler):
  for idx, batch in enumerate(tqdm(loader)):
    images, labels, geo_inf = batch
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    with torch.cuda.amp.autocast():
      predictions = model(images)
      loss = loss_function(predictions, labels)

    optimizer.zero_grad()
    scaler(loss).backward()
    scaler.step(optimizer)
    scaler.update()

In [14]:
def main():
  train_transform = albumentations.Compose([
      albumentations.Normalize(mean=total_mean, std=total_std, max_pixel_value=255.0),
      albumentations.ToTensorV2()
  ])
  val_transform = albumentations.Compose([
      albumentations.Normalize(mean=total_mean, std=total_std, max_pixel_value=255.0),
      albumentations.ToTensorV2()
  ])
  test_transform = albumentations.Compose([
      albumentations.Normalize(mean=total_mean, std=total_std, max_pixel_value=255.0),
      albumentations.ToTensorV2()
  ])
  model = UNet(in_channels=4, out_channels=6).to(DEVICE)
  loss_function = nn.CrossEntropyLoss()

In [15]:
if __name__ == "__main__":
  main()

AttributeError: module 'albumentations' has no attribute 'ToTensorV2'